# Facility Location demo

Outline:
1. Data
2. Model
3. Analysis

Input:
Output:


# Data

In [ ]:
# import custom functions related to our model
import model
import data_preparation
# import libraries
import pandas as pd

## Data Collection and Cleaning

In [ ]:
data_demand =pd.read_excel("C:/Data/Major city data.xlsx")


In [ ]:
# Show the data map (input of the model)


## Mainland USA and its demands

## Input Parameters of the model

# Modeling

## Model 1

In [ ]:
# Show the model equation


## Model 2

# Analysis

## Resource optimization


## Final Geographic map showing optimized allocation of facilities